In [5]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
import json
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from utils import flattenStructure
with open('../dataset/semeval2017-task8/' + 'train' + 'Set.json', 'r') as f:
            content = f.read()
rawDataset = json.loads(content)

In [14]:
word2vec = KeyedVectors.load_word2vec_format('../dataset/glove/glove.twitter.27B.25d.gensim.txt', binary=False)

In [18]:
dataset = []

for threadId in rawDataset['threadIds']:
    thread = []
    stanceTag = []
    structure = rawDataset['structures'][threadId]
    ids = flattenStructure(structure)
    for id in ids:
        if id in rawDataset['posts'] and id in rawDataset['stanceTag']:
            thread.append([id, rawDataset['posts'][id]['text']])
            stanceTag.append(rawDataset['label2IndexStance'][rawDataset['stanceTag'][id]])
    for i in range(len(thread)):
        text = thread[i][1]
        textW2v = []
        for word in text:
            if word in word2vec:
                textW2v.append(word2vec[word].tolist())
            else:
                textW2v.append([0. for _ in range(25)])
        thread[i][1] = torch.Tensor(textW2v)
    dataset.append({
        'threadId': id,
        'threads': thread,
        'rumorTag': torch.LongTensor(
            [rawDataset['label2IndexRumor'][rawDataset['rumorTag'][threadId]]]),
        'stanveTag': torch.LongTensor(stanceTag),
        'structure': structure
    })

In [19]:
dataset[0]

{'threadId': '498323335965868032',
 'threads': [['498280126254428160',
   tensor([[ 0.4677,  0.2444, -0.7976,  ...,  0.5102, -0.0215,  0.2422],
           [-0.2608,  0.5911,  0.6162,  ...,  0.0769,  0.2284,  0.2758],
           [ 1.3463, -0.1862,  0.2351,  ...,  0.4351,  1.7783, -0.4791],
           ...,
           [ 0.7121, -0.1883, -0.8360,  ...,  0.4831,  0.4100,  0.0061],
           [ 1.3635,  0.0401,  0.8293,  ...,  0.2410,  0.8334,  1.1436],
           [ 0.5323, -0.3042, -0.6065,  ...,  0.9421,  0.3408,  0.6696]])],
  ['498280920227794945',
   tensor([[-9.1012e-01, -2.1600e-01,  5.5683e-01, -5.5125e-01, -8.8005e-02,
             1.1302e-01,  9.0393e-01,  1.2515e+00,  4.9890e-01,  2.3189e-01,
            -3.5661e-01, -3.1751e-01, -2.9269e+00, -2.1685e-01, -3.3888e-01,
            -1.8477e+00,  2.9851e-03, -1.4393e-01, -1.1331e+00,  1.9579e-01,
            -2.5747e-01, -1.5152e+00,  3.7440e-02, -4.6161e-01,  1.1892e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+0